In [3]:
from __future__ import division
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv

try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

In [4]:
##################################### DATA INPUT BLOCK ########################################3

#read your file
file=r'Dict_9.tiff'
img_all = cv2.imread(file,0)
print 'Shape of the input image:',img_all.shape

#read templates for all of the numbers and minus sign
templates = []
number_names=['0','1','2','3','4','5','6','7','8','9','-']
for number in number_names:
    name = number + '.png'
    templates.append(cv2.imread(name,0))

# Cut only the right part of the page
img = img_all[:,1200:]

# Save it
cv2.imwrite('Dict_9_half.jpg',img)

Shape of the input image: (1653, 2336)


True

In [5]:
############################################ HELPER FUNCTIONS BLOCK #############################

# Distance calculation 
def dist(a,b,c,d):
    return abs(a-c) + abs(b-d)


# Combination of the line segments into the full width lines
def combineHLines(lines, threshold):
    newLines = []
    sortLines = []
    #Sort the lines
    for line in lines:
        for x1,y1,x2,y2 in line:
            # sort all lines from left to right
            if x1>x2:
                x_mem =  x1
                y_mem = y1
                x1 = x2
                y1 = y2
                x2 = x_mem
                y2 = y_mem
            sortLines.append([x1,y1,x2,y2])
            # if the line starts from the begining of the line
            if (x1>150) and (x1<170):
                newLines.append([x1,y1,x2,y2])
    # extendlines
    for i in range(len(newLines)):
        line = newLines[i]
        x1 = line[0]
        y1 = line[1]
        x2 = line[2]
        y2 = line[3]
        Expandable = True
        while Expandable and x2<940:
            Expandable = False
            minDist = 1000
            for lineS in sortLines:
                xi1 = lineS[0]
                yi1 = lineS[1]
                xi2 = lineS[2]
                yi2 = lineS[3]
                if dist(xi1,yi1,x2,y2)<minDist:
                    minDist = dist(xi1,yi1,x2,y2)
                    line_mem = lineS
            if minDist < threshold:
                x2 = line_mem[2]
                y2 = line_mem[3]
                Expandable = True
                newLines[i][2] = x2
                newLines[i][3] = y2
                
    #delete unfinished lines
    finalLines = []
    for line in newLines:
        if line[2] > 940:
            finalLines.append(line)
    return finalLines

# Sorting of countours and generation of bounding boxes
def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0
    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)


# Get the sub cell coordinates for the number
def getNumberFrame(yo,y1,x0,x1):
    frame = img[y0:y1,x0:x1]
    thresh, frame_T = cv2.threshold(frame,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return frame_T, 1

In [9]:
###################################### DESKEW BLOCK ##############################################

#thresholding the image to a binary image
thresh,img_bin = cv2.threshold(img,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

#inverting the image
img_bin = 255-img_bin

# countcol(width) of kernel as 100th of total width
kernel_len = np.array(img).shape[1]//100

# Defining a horizontal kernel to detect all horizontal lines of image
hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))

#Use horizontal kernel to detect and save the horizontal lines in a jpg
image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

# Apply edge detection method on the image 
edges = cv2.Canny(horizontal_lines,50,150,apertureSize = 3)
  
rho = 1  # distance resolution in pixels of the Hough grid
theta = np.pi / 180  # angular resolution in radians of the Hough grid
threshold = 15  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 50  # minimum number of pixels making up a line
max_line_gap = 20  # maximum gap in pixels between connectable line segments
line_image = np.copy(img) * 0  # creating a blank to draw lines on

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                    min_line_length, max_line_gap)

# Filter the lines to get full width lines
newLines = combineHLines(lines,60)

# collect angles
angles = []
for line in newLines:
    x1 = line[0]
    y1 = line[1]
    x2 = line[2]
    y2 = line[3]
    angle = float((y2-y1)/(x2-x1))
    angles.append(angle)
    cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)
             
# Save line detection output
cv2.imwrite('Dict_9_line_detection.jpg',line_image) 
         
angle = np.mean(angles)*180/np.pi
print 'Detected skew to be', angle, 'degrees'

# rotate the image to deskew it
(h, w) = img.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle, 1.0)
img_deskew = cv2.warpAffine(img, M, (w, h),
                         flags=cv2.INTER_CUBIC, 
                         borderMode=cv2.BORDER_REPLICATE)

# Save it
cv2.imwrite('Dict_9_half_rotated.jpg',img_deskew)

Detected skew to be -0.35311556856242077 degrees


True

In [10]:
########################## CELL DETECTION BLOCK #########################################################

# thresholding the image to a binary image
thresh,img_bin = cv2.threshold(img_deskew,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# inverting the image
img_bin = 255-img_bin

# countcol(width) of kernel as 100th of total width
kernel_len = np.array(img).shape[1]//100

# Defining a vertical kernel to detect all vertical lines of image
ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))

# Defining a horizontal kernel to detect all horizontal lines of image
hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))

# A kernel of 2x2
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

# Use vertical kernel to detect and save the vertical lines in a jpg
image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)

# Use horizontal kernel to detect and save the horizontal lines in a jpg
image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

# Combine horizontal and vertical lines in a new third image, with both having same weight.
img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)

# Eroding and thesholding the image
img_vh = cv2.erode(~img_vh, kernel, iterations=2)
thresh, img_vh = cv2.threshold(img_vh,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#cv2.imwrite("Dict_9_cell_detection.jpg", img_vh)
bitxor = cv2.bitwise_xor(img,img_vh)
bitnot = cv2.bitwise_not(bitxor)

#Plotting the generated image
cv2.imwrite("Dict_9_cell_detection.jpg", img_vh)

# Detect contours for following box detection
contours, hierarchy = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Sort all the contours by top to bottom.
contours, boundingBoxes = sort_contours(contours, method="top-to-bottom")

# Filter contours
c_filtered = []
width = []
height = []
for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    if (w>70 and w<80 and h>60):
        c_filtered.append([x,y,w,h])
        width.append(w)
        height.append(h)
        
print 'Average cell height:',np.mean(height)
print 'Average cell width:',np.mean(width)

Average cell height: 82.1
Average cell width: 75.7875


In [50]:
#########################################   OCR BLOCK   #########################################

# iterate through all detected cells
iterator = 0
container = []
for x,y,w,h in c_filtered:
    if iterator < 10:
        iterator += 1
        for i in range(3):
            nw = int(w / 3 + 4)
            y1 = y + nw * i
            y2 = y + nw * (i+1)-5
            x1 = x + 10
            x2 = x + w - 20
            frame = img_deskew[y1:y2,x+10:x+w-20]
            if i == 0:
                pFrame = frame
            else:
                print 'Iteration',iterator, i
                pFrame = np.hstack((pFrame,frame))
            cv2.imwrite(name, frame)
        name = 'Comb' + str(iterator) + '.jpg'
        cv2.imwrite(name, pFrame)
        primer = pytesseract.image_to_string(pFrame)
        container.append(primer)
        print primer

Iteration 1 1
Iteration 1 2
54 45 -38
Iteration 2 1
Iteration 2 2
59 40

-67
Iteration 3 1
Iteration 3 2
69 -56 31
Iteration 4 1
Iteration 4 2
42 56

-35
Iteration 5 1
Iteration 5 2
18 51 -11
Iteration 6 1
Iteration 6 2
34 55

-68
Iteration 7 1
Iteration 7 2
13 56 -19
Iteration 8 1
Iteration 8 2
49 50 -64
Iteration 9 1
Iteration 9 2

Iteration 10 1
Iteration 10 2
16 52
